In [1]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

### Get List Of All Countries From API

In [2]:
def get_all_countries():
    response = requests.get(
        'https://api.worldbank.org/v2/country',
        params={'format': 'json', 'per_page': 300}
    )
    if response.status_code == 200:
        return [
            country['name'] 
            for country in response.json()[1] 
            if country.get('regionCode') is None and country.get('capitalCity')
        ]
    return []

### Fetch indicator data for a specific country

In [3]:
def fetch_one_country(country, code, indicator, n_recent_years):
    try:
        response = requests.get(
            f'https://api.worldbank.org/v2/country/{code}/indicator/{indicator}',
            params={'format': 'json', 'per_page': 100}
        )
        if response.status_code != 200:
            return []
        
        data = response.json()[1]
        
        # filter for n_recent_years
        valid_data = [entry for entry in data if entry.get('value') is not None]
        recent_data = sorted(
            valid_data,
            key=lambda x: x['date'],
            reverse=True
        )[:n_recent_years]
        
        # Format the results
        return [{
            'country': country,
            'iso_3': code,
            'year': int(entry['date']),
            'value': entry['value']
        } for entry in recent_data]
    except Exception:
        return []

### Fetch data for indicators - returns a dictionary of DFs, one for each indicator

In [4]:
def fetch_indicators_data(indicators, countries, n_recent_years=3):
    
    # get country codes
    response = requests.get(
        'https://api.worldbank.org/v2/country',
        params={'format': 'json', 'per_page': 300}
    )
    code_lookup = {
        entity['name'].lower(): entity['id']
        for entity in response.json()[1]
    }
    
    # handle single country case
    if isinstance(countries, str):
        countries = [countries]
    
    # dictionary to store results
    indicator_dfs = {}
    
    # process each indicator
    for indicator in indicators:
        # prepare arguments for parallel processing
        tasks = [
            (country, code_lookup.get(country.lower()), indicator, n_recent_years)
            for country in countries
            if code_lookup.get(country.lower())  # skip if no valid code
        ]
        
        # fetch data in parallel using threads
        results = []
        with ThreadPoolExecutor(max_workers=25) as executor:
            futures = [
                executor.submit(fetch_one_country, country, code, ind, years)
                for country, code, ind, years in tasks
            ]
            for future in futures:
                country_data = future.result()
                if country_data:
                    results.extend(country_data)
        
        # create df if we got results
        if results:
            indicator_dfs[indicator] = pd.DataFrame(results)
    
    return indicator_dfs

### Main function 
Enter desired indicators here, they will be returned as dataframes.
**In order to have them recognized in Power BI, you must assign them to a variable!**
For example *_gdp_df = indicator_dataframes.get('NY.GDP.MKTP.CD')*

You can also use Pandas to save as a csv (df.to_csv('indicator.csv')

In [7]:
def get_world_bank_data():
    indicators = [
        'NY.GDP.MKTP.CD',  # GDP (current US$)
        # add more indicators here as needed
    ]
    
    countries = get_all_countries()
    return fetch_indicators_data(indicators, countries, n_recent_years=3)

# call the above function
indicator_dataframes = get_world_bank_data()
# the gdp dataframe will be available using the following line 
gdp_df = indicator_dataframes.get('NY.GDP.MKTP.CD')
gdp_df.to_csv('GDP_current_us.csv', index = False)